Objectives

- Combine two tables by concatenation
- Combine two tables by left join when tables share the same key column
- Combine two tables by left join by defining the columns to join on specifically

Content to cover

- pd.concat
- pd.merge


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Air quality data about $NO_2$ and particulate matter less than 2.5 micrometers is used, made available by [openaq](https://openaq.org) and using the [py-openaq](http://dhhagan.github.io/py-openaq/index.html) package:

- The `air_quality_no2_long.csv"` data set provides $NO_2$ values for the measurement stations _FR04014_, _BETR801_ and _London Westminster_ in respectively Paris, Antwerp and London. 
- The `air_quality_pm25_long.csv` data set provides $pm25$ values for the measurement stations _FR04014_, _BETR801_ and _London Westminster_ in respectively Paris, Antwerp and London. 
- The metadata about these stations is stored in a data file `air_quality_stations.csv`
- The metadata about the measured parameters is stored in a data file `air_quality_parameters.csv`

In [4]:
air_quality_no2 = pd.read_csv("../data/air_quality_no2_long.csv", 
                              parse_dates=True)
air_quality_no2.head()

,city,country,date.utc,location,parameter,value,unit
0,Paris,FR,2019-06-21 00:00:00+00:00,FR04014,no2,20.0,µg/m³
1,Paris,FR,2019-06-20 23:00:00+00:00,FR04014,no2,21.8,µg/m³
2,Paris,FR,2019-06-20 22:00:00+00:00,FR04014,no2,26.5,µg/m³
3,Paris,FR,2019-06-20 21:00:00+00:00,FR04014,no2,24.9,µg/m³
4,Paris,FR,2019-06-20 20:00:00+00:00,FR04014,no2,21.4,µg/m³


In [5]:
air_quality_pm25 = pd.read_csv("../data/air_quality_pm25_long.csv", 
                               parse_dates=True)
air_quality_pm25.head()

,city,country,date.utc,location,parameter,value,unit
0,Antwerpen,BE,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,µg/m³
1,Antwerpen,BE,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,µg/m³
2,Antwerpen,BE,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,µg/m³
3,Antwerpen,BE,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,µg/m³
4,Antwerpen,BE,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,µg/m³


In [156]:
air_quality_stations = pd.read_csv("../data/air_quality_stations.csv")
air_quality_stations.head()

,city,coordinates.latitude,coordinates.longitude,count,country,distance,firstUpdated,lastUpdated,location,parameters,sourceName,sourceNames
0,Antwerpen,51.236195,4.385224,4179,BE,5902293,2017-09-22 01:00:00+00:00,2019-08-05 03:00:00+00:00,BELAL01,"['pm10', 'pm25']",EEA Belgium,['EEA Belgium']
1,Antwerpen,51.170300,4.341005,8052,BE,5902428,2017-09-22 01:00:00+00:00,2019-08-05 03:00:00+00:00,BELHB23,"['so2', 'pm10', 'no2', 'pm25']",EEA Belgium,['EEA Belgium']
2,Antwerpen,51.109978,5.004864,1641,BE,5947480,2017-09-22 01:00:00+00:00,2019-01-09 01:00:00+00:00,BELLD01,['no2'],EEA Belgium,['EEA Belgium']
3,Antwerpen,51.120384,5.021546,1973,BE,5948067,2017-09-22 01:00:00+00:00,2019-08-05 03:00:00+00:00,BELLD02,['no2'],EEA Belgium,['EEA Belgium']
4,Antwerpen,51.327660,4.362261,1923,BE,5896736,2017-09-23 01:00:00+00:00,2019-08-05 03:00:00+00:00,BELR833,['no2'],EEA Belgium,['EEA Belgium']


In [157]:
air_quality_parameters = pd.read_csv("../data/air_quality_parameters.csv")
air_quality_parameters

,description,id,name,preferredUnit
0,Black Carbon,bc,BC,µg/m³
1,Carbon Monoxide,co,CO,ppm
2,Nitrogen Dioxide,no2,NO2,ppm
3,Ozone,o3,O3,ppm
4,Particulate matter less than 10 micrometers in...,pm10,PM10,µg/m³
5,Particulate matter less than 2.5 micrometers i...,pm25,PM2.5,µg/m³
6,Sulfur Dioxide,so2,SO2,ppm


## Combine data from multiple tables

### Concatenating objects

![](../schemas/08_concat_row.svg)

> I want to combine the measurements of $NO_2$ and $pm25$, two tables with a similar structure, in a single table

In [158]:
air_quality = pd.concat([air_quality_pm25, air_quality_no2])
air_quality.head()

,city,country,date.utc,location,parameter,value,unit
0,Antwerpen,BE,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0,µg/m³
1,Antwerpen,BE,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5,µg/m³
2,Antwerpen,BE,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5,µg/m³
3,Antwerpen,BE,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0,µg/m³
4,Antwerpen,BE,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5,µg/m³


The `concat` function performs concatenation operations of multiple tables along one of the axis (row-wise or column-wise). By default concatenation is along axis 0, so the resulting table combines the rows of the input tables. Let's check the shape of the original and the concatenated tables to verify the operation:

In [159]:
air_quality_pm25.shape, air_quality_no2.shape, air_quality.shape

((1110, 7), (2068, 7), (3178, 7))

Hence, the resulting table has 3178 = 1110 + 2068 rows.

Sorting the table on the datetime information illustrates also the combination of both tables, with the `parameter` column defining the origin of the table (either `no2` from table `air_quality_no2` or `pm25` from table `air_quality_pm25`):

In [160]:
air_quality = air_quality.sort_values("date.utc")
air_quality.head()

,city,country,date.utc,location,parameter,value,unit
2067,London,GB,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,µg/m³
1003,Paris,FR,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,µg/m³
100,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,µg/m³
1098,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,µg/m³
1109,London,GB,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0,µg/m³


In this specific example, the `parameter` column provided by the data ensures that each of the original tables can be identified. This is not always the case. the `concat` function provides a convenient solution with the `keys` argument, adding an additional (hierarchical) row index. For example:

In [161]:
air_quality_ = pd.concat([air_quality_pm25, air_quality_no2], keys=["PM25", "NO2"])

In [162]:
air_quality_.head()

city country                   date.utc location parameter  \
PM25 0  Antwerpen      BE  2019-06-18 06:00:00+00:00  BETR801      pm25   
     1  Antwerpen      BE  2019-06-17 08:00:00+00:00  BETR801      pm25   
     2  Antwerpen      BE  2019-06-17 07:00:00+00:00  BETR801      pm25   
     3  Antwerpen      BE  2019-06-17 06:00:00+00:00  BETR801      pm25   
     4  Antwerpen      BE  2019-06-17 05:00:00+00:00  BETR801      pm25   

        value   unit  
PM25 0   18.0  µg/m³  
     1    6.5  µg/m³  
     2   18.5  µg/m³  
     3   16.0  µg/m³  
     4    7.5  µg/m³

<div class="alert alert-info">
    
__Note__: The existence of multiple row/column indices at the same time has not been mentioned within these tutorials. _Hierarchical indexing_ or _MultiIndex_ is an advanced and powerfull Pandas feature to analyze higher dimensional data. 

Multi-indexing is out of scope for this Pandas introduction. For the moment, remember that the function `reset_index` can be used to convert any level of an index to a column, e.g. `air_quality.reset_index(level=0)`
    
__To user guide:__ Feel free to dive into the world of multi-indexing at :ref:`advanced`

</div>

__To user guide:__ More options on table concatenation (row and column wise) and how `concat` can be used to define the logic (union or intersection) of the indexes on the other axes 
is provided at :ref:`merging.concat`

### Join tables using a common identifier

![](../schemas/08_merge_left.svg)

> Add the station coordinates, provided by the stations metadata table, to the corresponding rows in the measurements table

First, extract the station location identifier and the coordinates from the `air_quality_stations` metadata table:

In [163]:
air_quality_stations_coord = air_quality_stations[["location", "coordinates.latitude", "coordinates.longitude"]]
air_quality_stations_coord.head()

,location,coordinates.latitude,coordinates.longitude
0,BELAL01,51.236195,4.385224
1,BELHB23,51.170300,4.341005
2,BELLD01,51.109978,5.004864
3,BELLD02,51.120384,5.021546
4,BELR833,51.327660,4.362261


__Note:__ The stations used in this example (FR04014, BETR801 and London Westminster) are just three entries enlisted in the metadata table. We only want to add the coordinates of these three to the measurements table, each on the corresponding rows of the `air_quality` table.

In [168]:
air_quality.head()

,city,country,date.utc,location,parameter,value,unit,coordinates.latitude,coordinates.longitude,id,description,name
0,London,GB,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,µg/m³,51.494670,-0.131931,no2,Nitrogen Dioxide,NO2
1,Paris,FR,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,µg/m³,48.837242,2.393903,no2,Nitrogen Dioxide,NO2
2,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,µg/m³,51.209663,4.431821,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
3,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,µg/m³,51.209663,4.431821,no2,Nitrogen Dioxide,NO2
4,London,GB,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0,µg/m³,51.494670,-0.131931,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5


In [165]:
air_quality = pd.merge(air_quality, air_quality_stations_coord, 
                       how='left', on='location')
air_quality.head()

,city,country,date.utc,location,parameter,value,unit,coordinates.latitude,coordinates.longitude
0,London,GB,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,µg/m³,51.494670,-0.131931
1,Paris,FR,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,µg/m³,48.837242,2.393903
2,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,µg/m³,51.209663,4.431821
3,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,µg/m³,51.209663,4.431821
4,London,GB,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0,µg/m³,51.494670,-0.131931


Using the `merge` function, for each of the rows in the `air_quality` table, the corresponding coordinates are added from the `air_quality_stations_coord` table. Both tables have the column `location` in common which is used as a key to combine the information. By choosing the `left` join, only the locations available in the `air_quality` (left) table, i.e. FR04014, BETR801 and London Westminster, end up in the resulting table. The `merge` function supports multiple join options similar to database-style operations. 

> Add the parameter full description and name, provided by the parameters metadata table, to the measurements table

In [169]:
air_quality_parameters_name = air_quality_parameters[['id','description', 'name']]
air_quality_parameters_name.head()

,id,description,name
0,bc,Black Carbon,BC
1,co,Carbon Monoxide,CO
2,no2,Nitrogen Dioxide,NO2
3,o3,Ozone,O3
4,pm10,Particulate matter less than 10 micrometers in...,PM10


In [170]:
air_quality = pd.merge(air_quality, air_quality_parameters_name, 
                       how='left', left_on='parameter', right_on='id')
air_quality.head()

,city,country,date.utc,location,parameter,value,unit,coordinates.latitude,coordinates.longitude,id_x,description_x,name_x,id_y,description_y,name_y
0,London,GB,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,µg/m³,51.494670,-0.131931,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
1,Paris,FR,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,µg/m³,48.837242,2.393903,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
2,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,µg/m³,51.209663,4.431821,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
3,Antwerpen,BE,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,µg/m³,51.209663,4.431821,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
4,London,GB,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0,µg/m³,51.494670,-0.131931,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5


Compared to the previous example, there is no common column name. However, the `parameter` column in the `air_quality` table and the `id` column in the `air_quality_parameters_name` both provide the measured variable in a common format. The `left_on` and `right_on` arguments are used here (instead of just `on`) to make the link between the two tables. 

 __To user guide:__ Pandas supports also inner, outer, and right joins. More information on join/merge of tables is provided in :ref:`merging.join`. Or have a look to the :ref:`comparison with SQL<compare_with_sql.join>`.

## REMEMBER

- Mulitple tables can be concatentated both column as row wise using the `concat` function.
- For database-like merging/joining of tables, use the `merge` function. 

__To user guide:__ The user guide provides more information on combining together data tables, see :ref:`merging`.